In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_palette("colorblind")
import os
import json
import urllib.request
import pickle
%matplotlib inline
fig = plt.figure(figsize=(20,5))

print(os.getcwd())
data = pd.read_csv('/Users/janechoi/Desktop/연구학점/credit card/data/funda_train.csv')

submission = pd.read_csv('/Users/janechoi/Desktop/연구학점/credit card/data/submission.csv')
#checking for NA values 
data.isnull().sum()  
#region              2042766
#type_of_business    3952609 : 전체 obs 중 대부분이 NA value 

#besides the two values clean dataset 

#delete both region and type-of-business since too much NA 
data.drop('region', axis=1, inplace=True)
data.drop('type_of_business',axis=1 , inplace=True)

#change data types 
data['store_id'] = data['store_id'].astype('str')   #store_id 2136 obs -categorical data
data['card_id'] = data['card_id'].astype('str')  #4658616 id's --doesn't really matter in the dataset 


#cut the year, month, day for EDA purpose 
data['year'] = data['transacted_date'].str[:4]
data['month'] = data['transacted_date'].str[5:7]
data['day'] = data['transacted_date'].str[8:]

#change transacted_date type 
data['transacted_date'] = pd.to_datetime(data['transacted_date'])

data[['transacted_date','year','month','day']]
data = data[data['amount']!=0]

#make id for store_id and card_id 
data['store_card'] = data['store_id'].astype(str) +'-' +data['card_id'].astype(str)
data.head()


data.head()


In [ ]:
#deleting the amount that have been refunded :: 
    # use date only before the 

#select amount data that is under 0 
minus = data[data['amount']< 0] 


#indexes we need to delete 
delete= []
notfound = []
#iterate the index for the minus data : minus.index 
#[     41,     347,     731,     831,     944,    1953,    2659, 2734,    3256,    3544, ..]

k=0
for i in minus.index : 
    
    #use minus amount dataset 
    
        #find minus index's store card and amount 
    find = data.loc[i,'store_card']
    amount = data.loc[i,'amount']
#     print(amount)

#     print(find)
    
    #find from original dataset 
    
        #match with the original dataset where has same amount and store_card id 
    org = data[(data['store_card'] == str(find)) &(data['amount'] == -amount)]
    
        #use first one if there is multiple : since it should be smaller than the index 
#     print(org.index)
#     print(i) 


        #there are some data where there is no match  
    try:
        #if there is a match : use the -1 index :: last data 
       
        ind = org.index[0]

        j=0
        while ind  in delete :
            print('inside while'+ str(j))
            j+=1
            if j < org.shape[0] : #row length 
                ind =org.index[j] #org's jth index 
            else:
                break #go to except 
            
        delete.append(i)
        delete.append(ind)
        
        
    except: 
        #no match in dataset append index to notfound list 
        print('exception')
        notfound.append(i)
        
    print(k, end='') #73100 iteration 
    k+=1
    




In [ ]:
import pickle #save list 

with open('delete_index2.txt', 'wb') as f:
    pickle.dump(delete, f)
    

with open('notfound_index2.txt', 'wb') as f:
    pickle.dump(notfound, f)

#open by... 
# with open('delete_index.txt', 'rb') as f: 
#     n = pickle.load(f) 
# type(n) #list

len(delete) #146080 
# len(notfound) #60





# drop the values 

dropped = data.drop(index=delete)
len(dropped[dropped['amount']<0]) #60 obs since match not found has 60 obs 

#delete values that have no match in the dataset too 
# dropped = dropped.drop(index=notfound)
# len(dropped[dropped['amount']<0])